In [5]:
import os
import cv2,collections
import sys
import math
import random
import numpy as np
import plotly.express as px
from queue import Queue
import matplotlib.pyplot as plt
sys.setrecursionlimit(1000000)

In [4]:
video_name="2016_04_07_07_09_21FrontColor_I"
directory="../images/"+video_name
if not os.path.exists(directory):
    os.makedirs(directory)
directoryf="../images/"+video_name+"/frames"
if not os.path.exists(directoryf):
    os.makedirs(directoryf)
directorym="../images/"+video_name+"/motion_vec"
if not os.path.exists(directorym):
    os.makedirs(directorym)
directoryj="../images/"+video_name+"/joined"
if not os.path.exists(directoryj):
    os.makedirs(directoryj)
directoryb="../images/"+video_name+"/joined_bl"
if not os.path.exists(directoryb):
    os.makedirs(directoryb)

In [ ]:
vidObj = cv2.VideoCapture("../videos/IN/{}.avi".format(video_name))
count = 0
success = 1
sec = 0
while success:
    vidObj.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    success, image = vidObj.read()
    if success == 1:
        cv2.imwrite(directoryf+"/frame%d.jpg" % count, image)
    count += 1
    sec += 0.1